In [21]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import pandas as pd
import altair as alt

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

def get_artist_id_by_name(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]['id']
    else:
        return None

def get_album_id_by_name(name):
    results = sp.search(q='album:' + name, type='album')
    items = results['albums']['items']
    if len(items) > 0:
        return items[0]['id']
    else:
        return None
    
def get_artist_info_by_name(name):
    results=sp.artist(get_artist_id_by_name(name))
    key=['name','genres','followers','popularity']
    value=[results['name'],results['genres'],results['followers']['total'],results['popularity']]
    my_dict={'info':key,'detail':value}
    return pd.DataFrame.from_dict(my_dict)
    
def get_album_tracks_by_name(name):
    results=sp.album(get_album_id_by_name(name))
    tracks=results['tracks']['items']
    print("The number of tracks in this album is:",results['total_tracks'])
    print("track list:")
    i=1
    for track in tracks:
        print("%s. %s"%(i,track['name']))
        i=i+1

def get_artist_albums_by_name(name,limit=10):
    results=sp.artist_albums(get_artist_id_by_name(name))
    albums=results['items']
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    name_lst=[]
    release_date=[]
    tracks_num=[]
    for album in albums:
        name_lst.append(album['name'])
        release_date.append(album['release_date'])
        tracks_num.append(album['total_tracks'])
    album_dict={'Album Name':name_lst,'Track Numbers':tracks_num,'Release Date':release_date}
    my_data=pd.DataFrame.from_dict(album_dict)
    return my_data.drop_duplicates(subset=['Album Name'],keep='first').sort_values(by='Release Date',ascending=False).head(limit)

def get_artist_top_tracks_by_name(name):
    results=sp.artist_top_tracks(get_artist_id_by_name(name))
    tracks=results['tracks']
    name_lst=[]
    popularity=[]
    Album_name=[]
    date_lst=[]
    for item in tracks:
        name_lst.append(item['name'])
        popularity.append(item['popularity'])
        Album_name.append(item['album']['name'])
        date_lst.append(item['album']['release_date'])
    track_dict={'Track name':name_lst,'Popularity':popularity,'Album':Album_name,'release date':date_lst}
    my_data=pd.DataFrame.from_dict(track_dict)
    return my_data.sort_values(by='Popularity',ascending=False)

def get_artist_track_numbers_for_years(name):
    results=sp.artist_albums(get_artist_id_by_name(name))
    albums=results['items']
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    name_lst=[]
    release_date=[]
    tracks_num=[]
    for album in albums:
        name_lst.append(album['name'])
        release_date.append(album['release_date'])
        tracks_num.append(album['total_tracks'])
    album_dict={'Album Name':name_lst,'Track Numbers':tracks_num,'Release Date':release_date}
    my_data=pd.DataFrame.from_dict(album_dict)
    my_data=my_data.drop_duplicates(subset=['Album Name'],keep='first').sort_values(by='Release Date',ascending=False)
    my_data['Release Date']=pd.to_datetime(my_data['Release Date'])
    my_data=my_data.resample('Y',on='Release Date').sum().reset_index()
    return alt.Chart(my_data).mark_line().encode(x='Release Date:T',y='Track Numbers:Q')

In [14]:
get_artist_track_numbers_for_years('joji')

alt.Chart(...)

In [15]:
get_artist_id_by_name('joji')

'3MZsBdqDrRTJihTHQrO6Dq'

In [16]:
get_album_id_by_name('Birdy')

'1WGjSVIw0TVfbp5KrOFiP0'

In [17]:
get_artist_albums_by_name('joji',10)

,Album Name,Track Numbers,Release Date
0,SMITHEREENS,9,2022-11-04
28,NOW That's What I Call Music! Vol. 84,20,2022-10-28
8,YUKON (INTERLUDE),2,2022-08-26
29,Seasick,15,2022-08-12
32,Towards The Moon,6,2022-07-08
31,Xeno,15,2022-07-08
12,Glimpse of Us,1,2022-06-10
34,Thinking Bout You,1,2022-05-27
35,Fragments,12,2022-01-14
36,From You,5,2022-01-03


In [18]:
get_artist_info_by_name('joji')

,info,detail
0,name,Joji
1,genres,[viral pop]
2,followers,7971464
3,popularity,86


In [19]:
get_artist_top_tracks_by_name('joji')

,Track name,Popularity,Album,release date
1,Die For You,85,SMITHEREENS,2022-11-04
2,SLOW DANCING IN THE DARK,81,BALLADS 1,2018-10-26
0,Glimpse of Us,80,SMITHEREENS,2022-11-04
3,YEAH RIGHT,76,BALLADS 1,2018-10-26
4,worldstar money (interlude),76,In Tongues (Deluxe),2018-02-14
5,Feeling Like The End,73,SMITHEREENS,2022-11-04
6,Sanctuary,73,Nectar,2020-09-24
7,YUKON (INTERLUDE),71,SMITHEREENS,2022-11-04
8,1AM FREESTYLE,71,SMITHEREENS,2022-11-04
9,NIGHT RIDER,70,SMITHEREENS,2022-11-04


In [22]:
get_album_tracks_by_name('Birdy')

The number of tracks in this album is: 11
track list:
1. 1901
2. Skinny Love
3. People Help the People
4. White Winter Hymnal
5. The District Sleeps Alone Tonight
6. I'll Never Forget You
7. Young Blood
8. Shelter
9. Fire and Rain
10. Without a Word
11. Terrible Love
